In [6]:
import os
import json
import pandas as pd
import pathlib
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [10]:
# Imposta il percorso della cartella desiderata
folder = '/content/drive/My Drive/road02_ins/'

# Elenco dei file e delle cartelle nella cartella
elenco_file_cartella = os.listdir(cartella)

# Stampa l'elenco dei file e delle cartelle
print(elenco_file_cartella)

['.DS_Store', 'Pose', 'ColorImage', 'Label']


In [11]:
image_dir = folder + "ColorImage/Record001/Camera 5"
label_dir = folder + "Label/Record001/Camera 5"
colorImage_dir = "/ColorImage"
labelImage_dir = "/Label"

In [12]:
""" These labels are related to object, more on ApolloScape site """
columnValues = [0, 1, 17, 33, 161, 34, 162, 35, 163, 36, 164, 37, 165, 38, 166,
                39, 167, 40, 168, 49, 50, 65, 66, 67, 81, 82, 83, 84, 85, 86, 97,
                98, 99, 100, 113, 255]

image = []
rows = []

In [14]:
""" Iterator to scroll the directories """
imageIterator = sorted(pathlib.Path(folder  + colorImage_dir).iterdir())
labelIterator = sorted(pathlib.Path(folder  + labelImage_dir).iterdir())

In [17]:
"""
    This function get the ID of every image contained in the folder
    (basically the name)
"""
def getPixelImages(path):
    for image_name in os.listdir(path):
            # print(image_name)
            image.append(str(path) + '/' + image_name)

In [18]:
print("*** IMAGES SECTION ***")

for path in imageIterator:
    if os.path.basename(path) == '.DS_Store':
        continue

    print(path)
    recordPath = sorted(pathlib.Path(path).iterdir())

    for record in recordPath:
        print("\n\n*** Elaborate Folder: " + str(record))
        if os.path.basename(record) == '.DS_Store':
            continue

        contentsPathSecond = sorted(pathlib.Path(record).iterdir())
        getPixelImages(record)


print("\nFinished elaborating images...")

*** IMAGES SECTION ***
/content/drive/My Drive/road02_ins/ColorImage/Record001


*** Elaborate Folder: /content/drive/My Drive/road02_ins/ColorImage/Record001/.DS_Store


*** Elaborate Folder: /content/drive/My Drive/road02_ins/ColorImage/Record001/Camera 5


*** Elaborate Folder: /content/drive/My Drive/road02_ins/ColorImage/Record001/Camera 6
/content/drive/My Drive/road02_ins/ColorImage/Record002


*** Elaborate Folder: /content/drive/My Drive/road02_ins/ColorImage/Record002/.DS_Store


*** Elaborate Folder: /content/drive/My Drive/road02_ins/ColorImage/Record002/Camera 5


*** Elaborate Folder: /content/drive/My Drive/road02_ins/ColorImage/Record002/Camera 6
/content/drive/My Drive/road02_ins/ColorImage/Record003


*** Elaborate Folder: /content/drive/My Drive/road02_ins/ColorImage/Record003/.DS_Store


*** Elaborate Folder: /content/drive/My Drive/road02_ins/ColorImage/Record003/Camera 5


*** Elaborate Folder: /content/drive/My Drive/road02_ins/ColorImage/Record003/Camera 6
/cont

In [19]:
"""
    This function get the Labels of every image and store into a dictionary
    composed by an ImageID and an array of Labels
"""
def getLabelsImages(path):

    for label in os.listdir(path):
            if label.endswith('json'):
                    with open(str(path) + '/' + label) as json_file:
                            data = json.load(json_file)
                            labels = []
                            for object in data['objects']:
                                    labels.append(object['label'])

                            labels = list(set(labels))
                            imageName = label.split('.')
                            row = {imageName[0]: labels}
                            rows.append(row)

In [24]:
print("*** LABELS SECTION ***")

for path in labelIterator:
    print(path)
    if os.path.basename(path) == '.DS_Store':
      continue


    recordPath = sorted(pathlib.Path(path).iterdir())

    for record in recordPath:
        print("\n\n*** Elaborate Folder: " + str(record))

        if os.path.basename(record) == '.DS_Store':
          continue

        contentsPathSecond = sorted(pathlib.Path(record).iterdir())
        getLabelsImages(record)

print("\nFinished elaborating labels...")

*** LABELS SECTION ***
/content/drive/My Drive/road02_ins/Label/.DS_Store
/content/drive/My Drive/road02_ins/Label/Record001


*** Elaborate Folder: /content/drive/My Drive/road02_ins/Label/Record001/Camera 5


*** Elaborate Folder: /content/drive/My Drive/road02_ins/Label/Record001/Camera 6
/content/drive/My Drive/road02_ins/Label/Record002


*** Elaborate Folder: /content/drive/My Drive/road02_ins/Label/Record002/Camera 5


*** Elaborate Folder: /content/drive/My Drive/road02_ins/Label/Record002/Camera 6
/content/drive/My Drive/road02_ins/Label/Record003


*** Elaborate Folder: /content/drive/My Drive/road02_ins/Label/Record003/Camera 5


*** Elaborate Folder: /content/drive/My Drive/road02_ins/Label/Record003/Camera 6
/content/drive/My Drive/road02_ins/Label/Record004


*** Elaborate Folder: /content/drive/My Drive/road02_ins/Label/Record004/.DS_Store


*** Elaborate Folder: /content/drive/My Drive/road02_ins/Label/Record004/Camera 5


*** Elaborate Folder: /content/drive/My Drive/r

In [25]:
"""
    This function create the final Dataframe comparing each image and seeing if
    it has its own JSON file for the label
"""
def createPartialDataframe():
    print("*** PARTIAL DATAFRAME CONSTRUCTION ***")
    tempDf = pd.DataFrame(columns=columnValues)

    for obj in image:
            tokensImg = obj.split('.')
            tokensImg.pop(-1)
            tokensImg = "".join(tokensImg)
            row = {'ImageID': obj}
            tokensImg2 = tokensImg.split('/')

            for label in rows:
                    if tokensImg2[-1] == list(label.keys())[0]:

                        arrayLabels = list(label[tokensImg2[-1]])
                        arrayLabels.sort()

                        for objectLabel in arrayLabels:
                            row.update({objectLabel: 1})

                        tempDf = tempDf.append(row, ignore_index=True)

    return tempDf

In [26]:
columnValues.sort()
df = pd.DataFrame(columns=columnValues)
df = createPartialDataframe()
print("\n\n*** Finished to construct total Dataframe ***")
df = df.fillna(0)

# Veryfing if exists a duplicated record
print("Total duplicates: " + str(df.ImageID.duplicated().sum()))

df.to_csv(folder + "dataset.csv", index=False)

*** PARTIAL DATAFRAME CONSTRUCTION ***


Streaming output truncated to the last 5000 lines.
<ipython-input-25-f39a3940c74f>:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tempDf = tempDf.append(row, ignore_index=True)
<ipython-input-25-f39a3940c74f>:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tempDf = tempDf.append(row, ignore_index=True)
<ipython-input-25-f39a3940c74f>:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tempDf = tempDf.append(row, ignore_index=True)
<ipython-input-25-f39a3940c74f>:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tempDf = tempDf.append(row, ignore_index=True)
<ipython-input-25-f39a3940c74f>:25: FutureWarning: The frame.append method is



*** Finished to construct total Dataframe ***
Total duplicates: 4020


In [31]:
df.to_csv("/content/drive/My Drive/dataset.csv", index=False)